In [74]:
import os
import spotipy
import spotipy.util as util
import time
import json
import csv
import pandas as pd
import sys
import pickle

In [75]:
scope = 'user-library-read'
username = 'your_spotify_username'
token = util.prompt_for_user_token(username,scope,client_id=os.environ['SPOTIPY_CLIENT_ID'],client_secret=os.environ['SPOTIPY_CLIENT_SECRET'],redirect_uri=os.environ['SPOTIPY_REDIRECT_URI'])
print(token)
sp = spotipy.Spotify(auth=token)

BQB5IWgdfhEEDPf8lncjAfPl7CjOqK6Cp5TGSfBQhtrXeYkabIGkgp7GJaGZB8gOo4MMpC6X_YTApRgQuii6bmAguaUmZ4U_fbL-NqbwPxXAQX4-HeaOlUgwjq3imTo53fuW01ilOwZMYuAYiKb9wzyv89Ag4lg


In [76]:
track_ids = []

In [77]:
with open('all_song_ids_no_dupes.csv') as csvfile:
    song_reader = csv.reader(csvfile, delimiter=',')
    for row in song_reader:
        track_ids.append(row[0])

In [78]:
print(track_ids[0:10])

['0HzbDl381rdXtSOaJk20eH', '7zSvbQuYJ4AhGLxsTRfCzv', '1MMp1H2Kib2BCDtdL5nL63', '1qylvO4iCIZZcqc4TqSjTZ', '6an3qGX5IcVW8I36JY1866', '4Hhv2vrOTy89HFRcjU3QOx', '0OhpOmbSSSlNROas9zGP7N', '5MT96Zz0ymUJNm8obKZQr0', '7z6t9FzbkML9y9iIxqqldr', '2rhX1xPAUJa7Lu1iiRfz3B']


In [79]:
features = sp.audio_features('4kV4N9D1iKVxx1KLvtTpjS')
analysis = sp.audio_analysis('4kV4N9D1iKVxx1KLvtTpjS')
print(analysis['meta'])

{'analyzer_version': '4.0.0', 'platform': 'Linux', 'detailed_status': 'OK', 'status_code': 0, 'timestamp': 1554004650, 'analysis_time': 5.82524, 'input_process': 'libvorbisfile L+R 44100->22050'}


In [80]:
def get_audio_features_and_analysis_from_song_id(sp, song_id):
    features = sp.audio_features(song_id)
    return features

In [81]:
def remove_string_keys_from_analysis(analysis):
    '''
    analysis: the dictionary returned from sp.audio_analysis(song_id)
    Remove the strings from the analysis because they take up a lot of space
    return: analysis but with said keys removed
    '''
    for key in ['codestring', 'echoprintstring', 'synchstring', 'rhythmstring']:
        analysis['track'].pop(key, None)
    return analysis

In [82]:
def save_song_features(songs_dictionary, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(songs_dictionary, f)

In [83]:
def get_audio_features_and_analysis_from_songs(sp, track_ids, start_index, end_index, block_size, consecutive_errors = 0, file_number = 1):
    '''
    sp: Spotipy Object
    track_ids: List of song ids
    start_index, end_index: only consider track_ids[start_index : end_index]
    return: a list of the form [ (track_id, features, analysis), ... ]
    '''
    if consecutive_errors > 25:
        print("Too many consecutive errors.")
        return
    try:
        song_features_and_analysis = {}
        for idx, track_id in enumerate(track_ids[start_index:end_index]):
            features = get_audio_features_and_analysis_from_song_id(sp, track_id)
            analysis = remove_string_keys_from_analysis(analysis)
            song_features_and_analysis[track_id] = (features, analysis)
            # Every 100 iterations Sleep for 5 seconds to not overload API
            # Also save progress along the way to a pickle
            if (idx+1) % block_size == 0:
                print("SONG ID IS: ", track_id)
                time.sleep(1)
                save_song_features(song_features_and_analysis, 'songs' + str(file_number))
                print("Songs left: ", end_index - (start_index+idx))
                song_features_and_analysis = {}
                start_index += block_size
                file_number += block_size
            consecutive_errors = 0
            
    except Exception as e:
        print(e)
        print("API called failed")
        print("Last song index was: ", start_index)
        print("Calling function again ")
        #Call the fucntion again from where we left off
        return get_audio_features_and_analysis_from_songs(sp, track_ids, start_index, end_index, consecutive_errors + 1, file_number)
    return 'Success'

In [84]:
def get_audio_features_from_songs_in_pickle(sp, track_ids, start_index, end_index, block_size, consecutive_errors = 0, file_number = 1):
    '''
    sp: Spotipy Object
    track_ids: List of song ids
    start_index, end_index: only consider track_ids[start_index : end_index]
    return: a list of the form [ (track_id, features, analysis), ... ]
    '''
    if consecutive_errors > 25:
        print("Too many consecutive errors.")
        return
    try:
        song_features = {}
        for idx, track_id in enumerate(track_ids[start_index:end_index]):
            features = get_audio_features_and_analysis_from_song_id(sp, track_id)
            song_features[track_id] = features
            # Every 100 iterations Sleep for 5 seconds to not overload API
            # Also save progress along the way to a pickle
            if (idx+1) % block_size == 0:
                print("SONG ID IS: ", track_id)
                time.sleep(1)
                save_song_features(song_features, 'songs' + str(file_number - 1))
                print("Songs left: ", end_index - (start_index+idx))
                song_features = {}
                start_index += block_size
                file_number += block_size
            consecutive_errors = 0
            
    except Exception as e:
        print(e)
        print("API called failed")
        print("Last song index was: ", start_index)
        print("Calling function again ")
        #Call the fucntion again from where we left off
        return get_audio_features_from_songs_in_pickle(sp, track_ids, start_index, end_index, consecutive_errors + 1, file_number)
    return 'Success'

In [85]:
# get_audio_features_from_songs_in_pickle(sp, track_ids, 0, 1000, 100)

In [88]:
print(len(track_ids))

63234


In [89]:
def get_audio_features_from_songs_in_json(sp, track_ids, start_index, end_index, consecutive_errors = 0, song_features = {}):
    '''
    sp: Spotipy Object
    track_ids: List of song ids
    start_index, end_index: only consider track_ids[start_index : end_index]
    return: a list of the form [ (track_id, features, analysis), ... ]
    '''
    if consecutive_errors > 25:
        print("Too many consecutive errors.")
        with open('incomplete_all_track_id_features.json','w') as outfile:
            json.dump(song_features, outfile)
        time.sleep(25)
        token = util.prompt_for_user_token(username,scope,client_id=os.environ['SPOTIPY_CLIENT_ID'],client_secret=os.environ['SPOTIPY_CLIENT_SECRET'],redirect_uri=os.environ['SPOTIPY_REDIRECT_URI'])
        print(token)
        sp = spotipy.Spotify(auth=token)
        print("new token")
        return get_audio_features_from_songs_in_json(sp, track_ids, start_index + 1, end_index, 0, song_features)
    try:
        for idx, track_id in enumerate(track_ids[start_index:end_index]):
            features = get_audio_features_and_analysis_from_song_id(sp, track_id)
            song_features[track_id] = features
            # Every 100 iterations Sleep for 5 seconds to not overload API
            # Also save progress along the way to a .csv
            if idx % 100 == 0:
                print("SONG ID IS: ", track_id)
                print("Songs left: ", end_index - start_index)
                # time.sleep(1)
                #with open('pop_track_id_features_and_analysis.json','a') as outfile:
                #  json.dump(song_features_and_analysis, outfile)
                # song_features_and_analysis = {}
                time.sleep(5)
            start_index += 1
            consecutive_errors = 0
            
    except Exception as e:
        print(e)
        print("API called failed")
        print("Last song index was: ", start_index)
        print("Calling function again ")
        #Call the function again from where we left off
        return get_audio_features_from_songs_in_json(sp, track_ids, start_index, end_index, consecutive_errors + 1, song_features)
    with open('all_track_id_features_0_63233.json','a') as outfile:
        json.dump(song_features, outfile)
    return 'Success'

In [90]:
get_audio_features_from_songs_in_json(sp, track_ids, 0, 63233, consecutive_errors = 0, song_features = {})

SONG ID IS:  0aqVhoenCvorCVcEZEfiLS
Songs left:  14233
SONG ID IS:  0VNzEY1G4GLqcNx5qaaTl6
Songs left:  14133
SONG ID IS:  0SxP0xk83sA6Du4wd3VF2K
Songs left:  14033
SONG ID IS:  2eMCoeD8MWC7sCTnMQTSkP
Songs left:  13933
SONG ID IS:  1enw4tlTh1WooUpJJ7ed8n
Songs left:  13833
SONG ID IS:  0BsFp2sJPOdtRg3oUVWuh8
Songs left:  13733
SONG ID IS:  6C5e9TBZbCPSIzR9krnoeT
Songs left:  13633
SONG ID IS:  0tQdLcD7uYJeeSReQqA3si
Songs left:  13533
SONG ID IS:  1uAjOQqTkOCtg6WgZNsmsH
Songs left:  13433
SONG ID IS:  4OmOyT28pVbWeIRCv09NPw
Songs left:  13333
SONG ID IS:  1xhgAa8MGi8Qd8T1i1d5oD
Songs left:  13233
SONG ID IS:  6qgCS8rnF8W0RFrmhhtx2r
Songs left:  13133
SONG ID IS:  7AQxzmkhwBJwgSYr0qQTVf
Songs left:  13033
SONG ID IS:  0QCUKSlSjhIP4j0VTzoMFd
Songs left:  12933
SONG ID IS:  4AP07Dho717JM5MgHRySQ8
Songs left:  12833
SONG ID IS:  17Zbj5tTucjCpIk2ooAfLS
Songs left:  12733
SONG ID IS:  24kLQBF5CBJyr0x06lRpiy
Songs left:  12633
SONG ID IS:  7d6O6cRlbRjW9RNTOoOXhm
Songs left:  12533
SONG ID IS

http status: 401, code:-1 - https://api.spotify.com/v1/audio-features/?ids=5KSvaTTXYxQrVRf9JefuXg:
 The access token expired
API called failed
Last song index was:  60501
Calling function again 
http status: 401, code:-1 - https://api.spotify.com/v1/audio-features/?ids=5KSvaTTXYxQrVRf9JefuXg:
 The access token expired
API called failed
Last song index was:  60501
Calling function again 
http status: 401, code:-1 - https://api.spotify.com/v1/audio-features/?ids=5KSvaTTXYxQrVRf9JefuXg:
 The access token expired
API called failed
Last song index was:  60501
Calling function again 
http status: 401, code:-1 - https://api.spotify.com/v1/audio-features/?ids=5KSvaTTXYxQrVRf9JefuXg:
 The access token expired
API called failed
Last song index was:  60501
Calling function again 
http status: 401, code:-1 - https://api.spotify.com/v1/audio-features/?ids=5KSvaTTXYxQrVRf9JefuXg:
 The access token expired
API called failed
Last song index was:  60501
Calling function again 
http status: 401, code:-1

'Success'